# Rotation and translation of a C60 molecule on a graphene sheet

![](demo.png)

## Table of Content <a name="TOC"></a>

1. [General setups](#setups)


2. [Define the auxiliary functions](#2.)   
    

3. [Calculations](#3.) 

 

### A. Learning objectives

- to learn different types of rotations of rigid bodies
- to combine rotations and translations while doing a PES scan


### B. Use cases

- rotational and translational scan of PES


### C. Functions

- `libra_py`  
  - `LoadMolecule`
    - [`Load_Molecule`](#Load_Molecule-1)
  - `LoadPT`
    - [`Load_PT`](#Load_PT-1)  
    
- `liblibra::libchemobjects`
  - `libchemsys`
    - `System`
      - [`System`](#System-1)
      - [`determine_functional_groups`](#determine_functional_groups-1)      
      - [`GROUP_ATOMS`](#GROUP_ATOMS-1)      
      - [`init_fragments`](#init_fragments-1)            
      - [`print_xyz`](#print_xyz-1)
      - [`ROTATE_FRAGMENT`](#ROTATE_FRAGMENT-1)
      

### D. Classes and class members

- `liblibra::libchemobjects`
  - `libchemsys`
    - `System`
      - [`System`](#System-1)
      - [`Atoms`](#Atoms-1)
      - [`Fragments`](#Fragments-1)
      - [`Number_of_atoms`](#Number_of_atoms-1)
      - [`Number_of_angles`](#Number_of_angles-1)
      - [`Number_of_bonds`](#Number_of_bonds-1)      
      - [`Number_of_dihedrals`](#Number_of_dihedrals-1)
      - [`Number_of_fragments`](#Number_of_fragments-1)
      - [`Number_of_impropers`](#Number_of_impropers-1)        
      
  - `libuniverse`
      - `Universe`
        - [`Universe`](#Universe-1)
    

## 1. General setups
<a name="setups"></a>[Back to TOC](#TOC)

Let's import all the needed libraries, including the `py3Dmol` library for visualizing the structures we generate.

In [1]:
import math
import liblibra_core
from liblibra_core import *
from libra_py import units
from libra_py import LoadPT, LoadMolecule

import os, sys
#stdout = sys.stdout

import py3Dmol
import matplotlib.pyplot as plt
#from matplotlib.pyplot import figure
%matplotlib inline

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}
colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::v

## 2. Define the auxiliary functions
<a name="2."></a>[Back to TOC](#TOC)

Here is the function to load the molecule and to print out the basic info.

Note that the format of the .pdb file we use in this example is different from those we have seen before. It goes like:

    CRYST1   22.104   21.270   30.000  90.00  90.00  90.00 P 1           1
    ATOM      1  C   MOL X   1      13.451  10.685  10.000  0.00  0.00            
    ATOM      2  C1  MOL X   1      13.451   9.315  10.000  0.00  0.00            
    ATOM      3  C2  MOL X   1       6.549  10.685  10.000  0.00  0.00            
    ATOM      4  C3  MOL X   1       6.549   9.315  10.000  0.00  0.00            
    ATOM      5  C4  MOL X   1      10.685  10.000  13.451  0.00  0.00            
    ...

The keyword to read this format is `true_pdb2`
<a name="Universe-1"></a><a name="Load_PT-1"></a><a name="Load_Molecule-1"></a><a name="System-1"></a><a name="determine_functional_groups-1"></a><a name="GROUP_ATOMS-1"></a><a name="init_fragments-1"></a><a name="Number_of_atoms-1"></a><a name="Number_of_bonds-1"></a><a name="Number_of_angles-1"></a><a name="Number_of_impropers-1"></a><a name="Number_of_dihedrals-1"></a><a name="Number_of_fragments-1"></a>

In [2]:
def load_mol():
    # Create Universe and populate it
    U = Universe(); LoadPT.Load_PT(U, os.getcwd()+"/elements.txt")

    # Create molecular system and initialize the properties
    syst0 = System();  LoadMolecule.Load_Molecule(U, syst0, "system.pdb", "true_pdb2")
    syst0.determine_functional_groups(0)  # do not assign rings

    # Define the groups: note - the indexing of atoms and fragments starts from 1, not 0
    syst0.GROUP_ATOMS( list(range(1,61)), 1)    # C60
    syst0.GROUP_ATOMS( list(range(61,241)), 2)  # graphene 

    syst0.init_fragments()
    print("Number of atoms in the system = ", syst0.Number_of_atoms)
    print("Number of bonds in the system = ", syst0.Number_of_bonds)
    print("Number of angles in the system = ", syst0.Number_of_angles)
    print("Number of dihedrals in the system = ", syst0.Number_of_dihedrals)
    print("Number of impropers in the system = ", syst0.Number_of_impropers)
    print("Number of fragments in the system = ", syst0.Number_of_fragments)

    return syst0


In [3]:
def animate_traj(filename):
    xyz = ""
    f = open(filename,"r")
    for a in f.readlines():
        xyz = xyz + a
    f.close()

    view = py3Dmol.view(width=800,height=400)  
    view.setBackgroundColor('0xeeeeee')                                     
    view.zoomTo()                                                           
    view.addModelsAsFrames(xyz, 'xyz')
    view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
    view.animate({'reps':0, 'loop':'forward', 'step':1})
    view.show()  

Next, we define various schedules for performing rotation, translations, and a schedule to combine both types of transformations
<a name="Fragments-1"></a><a name="Atoms-1"></a><a name="ROTATE_FRAGMENT-1"></a><a name="print_xyz-1"></a>

In [4]:
def test_rotations(case, do_animate=0):

    syst0 = load_mol()
     
    u, pivot_pt = None, None    

    if case==100:
        # Rotation around the center of mass around x axis
        pivot_pt = syst0.Fragments[0].Group_RB.rb_cm
        u = VECTOR(1.0, 0.0, 0.0)

    elif case==101:  
        # Rotation around the center of mass around y axis
        pivot_pt = syst0.Fragments[0].Group_RB.rb_cm
        u = VECTOR(0.0, 1.0, 0.0)

    elif case==102:  
        # Rotation around the center of mass around z axis
        pivot_pt = syst0.Fragments[0].Group_RB.rb_cm
        u = VECTOR(0.0, 0.0, 1.0)

        
    elif case==200:  
        # Rotation around an external center around x axis
        pivot_pt = VECTOR(0.0, 0.0, 0.0)
        u = VECTOR(1.0, 0.0, 0.0)

    elif case==201:  
        # Rotation around an external center around y axis
        pivot_pt = VECTOR(0.0, 0.0, 0.0)
        u = VECTOR(0.0, 1.0, 0.0)

    elif case==202:  
        # Rotation around an external center around y axis
        pivot_pt = VECTOR(0.0, 0.0, 0.0)
        u = VECTOR(0.0, 0.0, 1.0)


    elif case==300:  
        # Rotation around an atom around x axis
        pivot_pt = syst0.Atoms[0].Atom_RB.rb_cm
        u = VECTOR(1.0, 0.0, 0.0)

    elif case==301:  
        # Rotation around an atom around y axis
        pivot_pt = syst0.Atoms[0].Atom_RB.rb_cm
        u = VECTOR(0.0, 1.0, 0.0)

    elif case==302:  
        # Rotation around an atom around z axis
        pivot_pt = syst0.Atoms[0].Atom_RB.rb_cm
        u = VECTOR(0.0, 0.0, 1.0)


    elif case==400:  
        # Rotation around an atom around the bond between two atoms
        pivot_pt = syst0.Atoms[0].Atom_RB.rb_cm
        u = syst0.Atoms[0].Atom_RB.rb_cm - syst0.Atoms[1].Atom_RB.rb_cm


   
    filename = F"rot-case-{case}.xyz"
    f = open(filename,"w");  

    dphi = 1.0
    frag_id = 1

    for i in range(360): 
        syst0.ROTATE_FRAGMENT(dphi, u, frag_id,  pivot_pt)
        syst0.print_xyz(filename, i)
        
    f.close()  
    
    if do_animate==1:
        animate_traj(filename)
        
        



def test_translations(case, do_animate=0):

    syst0 = load_mol()
     
    u = None

    if case==100:
        u = VECTOR(1.0, 0.0, 0.0)

    elif case==101:
        u = VECTOR(0.0, 1.0, 0.0)

    elif case==102:
        u = VECTOR(0.0, 0.0, 1.0)


    filename = F"tr-case-{case}.xyz"
    f = open(filename,"w")  

    dr = 0.1
    frag_id = 1

    for i in range(100): 
        syst0.TRANSLATE_FRAGMENT(dr, u, frag_id)  
        syst0.print_xyz(filename, i)

    f.close()  
    
    if do_animate==1:
        animate_traj(filename)


def test_mixed(case, do_animate=0):

    syst0 = load_mol()
     
    u, pivot_pt, tdir = None, None, None    

    if case==100:
        u = VECTOR(0.0, 1.0, 0.0)
        tdir = VECTOR(1.0, 0.0, 0.0)

    filename = F"mixed-case-{case}.xyz"
    f = open(filename,"w")

    dphi = 3.6
    dr = 0.1
    frag_id = 1

    for i in range(100): 
        syst0.TRANSLATE_FRAGMENT(dr, tdir, frag_id)  
        pivot_pt = syst0.Fragments[0].Group_RB.rb_cm
        syst0.ROTATE_FRAGMENT(dphi, u, frag_id,  pivot_pt)

        syst0.print_xyz(filename, i)
        
    f.close()  

    if do_animate==1:
        animate_traj(filename)



## 3. Calculations
<a name="3."></a>[Back to TOC](#TOC)

Now, we are ready to run all the rotations:

In [5]:
for case in [100, 101, 102, 200, 201, 202, 300, 301, 302, 400]:
    test_rotations(case, 0)


242
Number of atoms in the system =  240
Number of bonds in the system =  0
Number of angles in the system =  0
Number of dihedrals in the system =  0
Number of impropers in the system =  0
Number of fragments in the system =  2
242
Number of atoms in the system =  240
Number of bonds in the system =  0
Number of angles in the system =  0
Number of dihedrals in the system =  0
Number of impropers in the system =  0
Number of fragments in the system =  2
242
Number of atoms in the system =  240
Number of bonds in the system =  0
Number of angles in the system =  0
Number of dihedrals in the system =  0
Number of impropers in the system =  0
Number of fragments in the system =  2
242
Number of atoms in the system =  240
Number of bonds in the system =  0
Number of angles in the system =  0
Number of dihedrals in the system =  0
Number of impropers in the system =  0
Number of fragments in the system =  2
242
Number of atoms in the system =  240
Number of bonds in the system =  0
Number o

translations

In [6]:
for case in [100, 101, 102]:
    test_translations(case, 0)


242
Number of atoms in the system =  240
Number of bonds in the system =  0
Number of angles in the system =  0
Number of dihedrals in the system =  0
Number of impropers in the system =  0
Number of fragments in the system =  2
242
Number of atoms in the system =  240
Number of bonds in the system =  0
Number of angles in the system =  0
Number of dihedrals in the system =  0
Number of impropers in the system =  0
Number of fragments in the system =  2
242
Number of atoms in the system =  240
Number of bonds in the system =  0
Number of angles in the system =  0
Number of dihedrals in the system =  0
Number of impropers in the system =  0
Number of fragments in the system =  2


and their combination

In [7]:
for case in [100]:
    test_mixed(case, 0)


242
Number of atoms in the system =  240
Number of bonds in the system =  0
Number of angles in the system =  0
Number of dihedrals in the system =  0
Number of impropers in the system =  0
Number of fragments in the system =  2
